# Occupation

### Introduction:

Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [1]:
import pandas as pd 
import numpy as np 

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user). 

### Step 3. Assign it to a variable called users.

In [2]:
users = pd.read_csv('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user', sep = '|')
users.head()

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


### Step 4. Discover what is the mean age per occupation

In [3]:
g = users.groupby('occupation').agg(np.mean)
g.age.sort_values()

occupation
student          22.081633
none             26.555556
entertainment    29.222222
artist           31.392857
homemaker        32.571429
programmer       33.121212
technician       33.148148
other            34.523810
scientist        35.548387
salesman         35.666667
writer           36.311111
engineer         36.388060
lawyer           36.750000
marketing        37.615385
executive        38.718750
administrator    38.746835
librarian        40.000000
healthcare       41.562500
educator         42.010526
doctor           43.571429
retired          63.071429
Name: age, dtype: float64

### Step 5. Discover the Male ratio per occupation and sort it from the most to the least

In [8]:

# first way, without pivot table
g = users.groupby(['occupation', 'gender']).count() # get male and female counts
occupations = g.index.get_level_values(0).unique() # get list of unique occupations
df_occ = pd.DataFrame([], occupations) # make data frame to store results
for i,occ in enumerate(occupations):
    try:
        males = g.loc[(occ, 'M'),'age']
    except KeyError:
        males = 0
    try:
        fems = g.loc[(occ, 'F'),'age']
    except KeyError:
        fems = 0
    
    if males == 0:
        df_occ.loc[occ, 'MFR'] = 0
    elif fems == 0:
        df_occ.loc[occ, 'MFR'] = np.Inf
    else:
        df_occ.loc[occ, 'MFR'] = males / fems

df_occ.MFR.sort_values()

user_id  age  zip_code
occupation    gender                        
administrator F            36   36        36
              M            43   43        43
artist        F            13   13        13
              M            15   15        15
doctor        M             7    7         7

In [57]:
# preferred way, with pivot table. Can easily set doctor values to zero, don't need to pre-allocate a data table, etc.
piv = users.pivot_table(index='occupation', columns='gender', aggfunc='size', fill_value=0) # compare aggfunct='count', size is what we want, counter-intuitively.
piv['MF_ratio'] = piv.M / piv.F
piv.sort_values(by = 'MF_ratio', axis = 0, inplace=True)
piv

gender,F,M,MF_ratio
occupation,,,
homemaker,6,1,0.166667
healthcare,11,5,0.454545
librarian,29,22,0.758621
artist,13,15,1.153846
administrator,36,43,1.194444
none,4,5,1.250000
writer,19,26,1.368421
marketing,10,16,1.600000
other,36,69,1.916667


### Step 6. For each occupation, calculate the minimum and maximum ages

In [56]:
# love this agg magic
t = users.groupby('occupation').agg(min_age = ('age', 'min'), max_age = ('age', 'max')).sort_values(by=['min_age', 'max_age'])

t

,min_age,max_age
occupation,,
student,7,42
none,11,55
other,13,64
entertainment,15,50
writer,18,60
salesman,18,66
artist,19,48
homemaker,20,50
programmer,20,63


### Step 7. For each combination of occupation and gender, calculate the mean age

In [60]:
mean_ages = users.groupby(['occupation', 'gender']).mean().age
mean_ages

occupation     gender
administrator  F         40.638889
               M         37.162791
artist         F         30.307692
               M         32.333333
doctor         M         43.571429
educator       F         39.115385
               M         43.101449
engineer       F         29.500000
               M         36.600000
entertainment  F         31.000000
               M         29.000000
executive      F         44.000000
               M         38.172414
healthcare     F         39.818182
               M         45.400000
homemaker      F         34.166667
               M         23.000000
lawyer         F         39.500000
               M         36.200000
librarian      F         40.000000
               M         40.000000
marketing      F         37.200000
               M         37.875000
none           F         36.500000
               M         18.600000
other          F         35.472222
               M         34.028986
programmer     F         32.16666

### Step 8.  For each occupation present the percentage of women and men

In [75]:
piv = users.pivot_table(index='occupation', columns='gender', aggfunc='size', fill_value=0) # compare aggfunct='count', size is what we want, counter-intuitively.
piv['total_users'] = piv['F'] + piv['M']
piv['percent_F'] = np.round(piv['F'] / piv['total_users'] * 100, 1)
piv['percent_M'] = np.round(piv['M'] / piv['total_users'] * 100,1)
piv = piv.drop('total_users', axis=1)
piv

gender,F,M,percent_F,percent_M
occupation,,,,
administrator,36,43,45.6,54.4
artist,13,15,46.4,53.6
doctor,0,7,0.0,100.0
educator,26,69,27.4,72.6
engineer,2,65,3.0,97.0
entertainment,2,16,11.1,88.9
executive,3,29,9.4,90.6
healthcare,11,5,68.8,31.2
homemaker,6,1,85.7,14.3
